In [ ]:
# import os
# import pandas as pd

# ##.dat 파일이 있는 디렉토리 지정
# dir_dat = '/Users/haseung-won/Desktop/학교/연구실/time_series_data/PAMAP2_Dataset/Optional/'
# dir_csv = '/Users/haseung-won/Desktop/학교/연구실/time_series_data/PAMAP2_Dataset/Optional_csv/'
# ##디렉토리 내의 모든 파일을 순회
# for filename in os.listdir(dir_dat):
#     ## .dat 파일인 경우만 처리
#     if filename.endswith('.dat'):
#         ## .dat 파일 읽기 (구분자가 공백인 경우)
#         data = pd.read_csv(os.path.join(dir_dat, filename), sep="\s+", header=None)

#         ## 새로운 .csv 파일 이름 생성 (.dat를 .csv로 변경)
#         new_filename = os.path.splitext(filename)[0] + '.csv'

#         ## .csv 파일로 저장
#         data.to_csv(os.path.join(dir_csv, new_filename), index=False, header=False) 


In [ ]:
import pandas as pd
df_list=[]
for subject_id in range(1,10,1):

    dir_csv = f'/Users/haseung-won/Desktop/학교/연구실/time_series_data/PAMAP2_Dataset/Protocol_csv/subject10{str(subject_id)}.csv'

    # Define column names based on the provided structure
    column_names = [
        'timestamp',
        'activityID',
        'heart_rate',
    ]

    imu_sensors = [
        'temperature',
        'acceleration16g_x', 'acceleration16g_y', 'acceleration16g_z',
        'acceleration6g_x', 'acceleration6g_y', 'acceleration6g_z',
        'gyroscope_x', 'gyroscope_y', 'gyroscope_z',
        'magnetometer_x', 'magnetometer_y', 'magnetometer_z',
        'orientation_1', 'orientation_2', 'orientation_3','orientation_4'
    ]

    imu_parts = ['hand', 'chest', 'ankle']

    # Add column names for IMU hand, chest, and ankle data
    for part in imu_parts:
        for sensor in imu_sensors:
            column_names.append(f'IMU_{part}_{sensor}')
    print(column_names)
    print(len(column_names))
    # Read the CSV file using pandas
    df = pd.read_csv(dir_csv, names=column_names)
    df_list.append(df)

    # Now df contains the data from the CSV file with appropriate column names


In [ ]:
import pandas as pd
test_df_list=[]
subject_id_list=[1,5,6,8,9]
for subject_id in subject_id_list:

    dir_csv = f'/Users/haseung-won/Desktop/학교/연구실/time_series_data/PAMAP2_Dataset/Optional_csv/subject10{str(subject_id)}.csv'

    # Define column names based on the provided structure
    column_names = [
        'timestamp',
        'activityID',
        'heart_rate',
    ]

    imu_sensors = [
        'temperature',
        'acceleration16g_x', 'acceleration16g_y', 'acceleration16g_z',
        'acceleration6g_x', 'acceleration6g_y', 'acceleration6g_z',
        'gyroscope_x', 'gyroscope_y', 'gyroscope_z',
        'magnetometer_x', 'magnetometer_y', 'magnetometer_z',
        'orientation_1', 'orientation_2', 'orientation_3','orientation_4'
    ]

    imu_parts = ['hand', 'chest', 'ankle']

    # Add column names for IMU hand, chest, and ankle data
    for part in imu_parts:
        for sensor in imu_sensors:
            column_names.append(f'IMU_{part}_{sensor}')
    print(column_names)
    print(len(column_names))
    # Read the CSV file using pandas
    df = pd.read_csv(dir_csv, names=column_names)
    test_df_list.append(df)

    # Now df contains the data from the CSV file with appropriate column names


# 데이터 시각화


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # 예시용 데이터 생성 (실제 데이터를 읽어오는 부분으로 대체해야 합니다)
# data = df_list[0]

# # 시간에 대한 열을 Datetime 형식으로 변환합니다.
# data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

# # 색상 맵 설정 (activityID에 맞게 색상 지정)
# num_activities = data['activityID'].nunique()
# cmap = plt.get_cmap('viridis', num_activities)

# # IMU 데이터 컬럼명 리스트
# imu_columns = data.columns[3:]

# plt.figure(figsize=(12, 8))

# for imu_column in imu_columns:
#     plt.figure(figsize=(12, 6))
    
#     for activity_id, group in data.groupby('activityID'):
#         plt.plot(group['timestamp'], group[imu_column], label=f'Activity {activity_id}', color=cmap(activity_id - 1))

#     plt.xlabel('Time')
#     plt.ylabel('Value')
#     plt.title(f'{imu_column} Time Series Data with Different Colors for ActivityID')
#     plt.legend(loc='upper right')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical

# 데이터 병합
data = df_list[0]


data = data.fillna(0)
X = data.drop(columns=['activityID'])
y = data['activityID']
num_classes =25
n_features = X.shape[1]  # Number of columns in your feature matrix X
print(num_classes)
# 시퀀스 길이 설정
timesteps = 10

X_train_lstm = []
y_train_lstm = []

for i in range(len(X) - timesteps + 1):
    X_sequence = X.iloc[i:i + timesteps, :].values
    y_label = y.iloc[i + timesteps - 1]  # Get a single label for the sequence
    X_train_lstm.append(X_sequence)
    y_train_lstm.append(y_label)


X_train_lstm = np.array(X_train_lstm)
y_train_lstm = np.array(y_train_lstm)
print(num_classes)
# Now you can proceed with multi-label encoding
y_train_encoded = np.array([to_categorical(labels, num_classes=num_classes) for labels in y_train_lstm])
y_train_encoded_flattened = y_train_encoded.reshape(-1, num_classes)

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(units=50, input_shape=(timesteps, n_features)))
model.add(Dense(units=num_classes, activation='sigmoid'))  # Use 'sigmoid' for multi-label classification

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'binary_crossentropy' for multi-label

# 모델 학습
epochs = 10
batch_size = 64
model.fit(X_train_lstm, y_train_encoded_flattened, epochs=epochs, batch_size=batch_size)





In [ ]:
model.save("model/Elstm_timeseries_model.h5")
print("Model saved successfully.")

In [ ]:
# Assuming df_list[1] contains your evaluation data

# Preprocessing on df_list[1]
eval_data = df_list[1].fillna(0)
X_eval = eval_data.drop(columns=['activityID'])
y_eval = eval_data['activityID']

# Prepare sequences and labels for evaluation
X_eval_lstm = []
y_eval_lstm = []

for i in range(len(X_eval) - timesteps + 1):
    X_sequence = X_eval.iloc[i:i + timesteps, :].values
    y_label = y_eval.iloc[i + timesteps - 1]  # Get a single label for the sequence
    X_eval_lstm.append(X_sequence)
    y_eval_lstm.append(y_label)

X_eval_lstm = np.array(X_eval_lstm)
y_eval_lstm = np.array(y_eval_lstm)

# One-hot encode the evaluation labels
y_eval_encoded = np.array([to_categorical(label, num_classes=num_classes) for label in y_eval_lstm])
y_eval_encoded_flattened = y_eval_encoded.reshape(-1, num_classes)

# Evaluate the model on the evaluation data
eval_loss, eval_accuracy = model.evaluate(X_eval_lstm, y_eval_encoded_flattened)

print("Evaluation Loss:", eval_loss)
print("Evaluation Accuracy:", eval_accuracy)


In [ ]:
import matplotlib.pyplot as plt

# Predict labels using the model
y_eval_pred_encoded_flattened = model.predict(X_eval_lstm)

# Inverse one-hot encode to get predicted labels
y_eval_pred_lstm = np.argmax(y_eval_pred_encoded_flattened, axis=1)

# Plot the actual and predicted labels over time
plt.figure(figsize=(12, 6))
plt.plot(y_eval_lstm, label='Actual Labels', marker='o')
plt.plot(y_eval_pred_lstm, label='Predicted Labels', marker='x')

# Get unique activityIDs
activity_ids = y_eval.unique()

# Find and annotate the ranges of each activityID
for activity_id in activity_ids:
    indices = np.where(y_eval_lstm == activity_id)[0]
    if indices.any():
        plt.annotate(f'Activity {activity_id}', 
                     xy=(indices[0], 0), 
                     xytext=(indices[0], -0.1), 
                     textcoords='offset points',
                     arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"),
                     fontsize=10)

plt.xlabel('Time Step')
plt.ylabel('Label')
plt.title('Actual vs. Predicted Labels')
plt.legend()
plt.show()
